## 3.1 EntropyMeasureV1

The instance `EntropyMeasure` contains 2 methods, one is `EntropyHadamardTest` using Hadamard test, and the other is `EntropyRandomizedMeasure` using randomized measure, the default case.

Both of them are used to calculate the entropy of a quantum state, but the calculation and post-processing are totally different.

`EntropyRandomizedMeasureV1` is a deprecated version of `EntropyRandomizedMeasure` because it lacks clarity on the relationship between bitstrings and qubit indices. 
This limitation can easily cause confusion, especially when working with systems that do not measure all qubits. However, it is retained for backward compatibility.

### 3.1.3 Randomized Measurement V1

This method is based on [Probing Rényi entanglement entropy via randomized measurements](https://www.science.org/doi/abs/10.1126/science.aau4963) with deplorizing error mitigation by [Simple mitigation of global depolarizing errors in quantum simulations](https://link.aps.org/doi/10.1103/PhysRevE.104.035309).


---

### a. `randomized_entangled_entropy_v1`

<a id='randomized_entangled_entropy_v1'></a>


Here, we assume you already have the data from the calculation. You can call the function `randomized_entangled_entropy_v1` from `qurry.process.randomized_measure` to calculate the entropy.

In this function, only `shots`, `counts`, and `degree` are required arguments. The others are optional.

The following is the arguments of the function:

```python
def randomized_entangled_entropy_v1(
    shots: int,
    counts: list[dict[str, int]],
    degree: Optional[Union[tuple[int, int], int]],
    measure: Optional[tuple[int, int]] = None,
    backend: PostProcessingBackendLabel = DEFAULT_PROCESS_BACKEND,
    workers_num: Optional[int] = None,
    pbar: Optional[tqdm.tqdm] = None,
) -> RandomizedEntangledEntropyComplex:
    """Calculate entangled entropy.

    Args:
        shots (int):
            Shots of the counts.
        counts (list[dict[str, int]]):
            Counts from randomized measurement results.
        degree (Optional[Union[tuple[int, int], int]]):
            The range of partition.
        measure (Optional[tuple[int, int]], optional):
            The range that implemented the measuring gate.
            If not specified, then use all qubits.
            This will affect the range of partition
            when you not implement the measuring gate on all qubit.
            Defaults to None.
        backend (PostProcessingBackendLabel, optional):
            Backend for the post-processing.
            Defaults to DEFAULT_PROCESS_BACKEND.
        workers_num (Optional[int], optional):
            Number of multi-processing workers, it will be ignored if backend is Rust.
            if sets to 1, then disable to using multi-processing;
            if not specified, then use the number of all cpu counts by `os.cpu_count()`.
            This only works for Python and Cython backend.
            Defaults to None.
        pbar (Optional[tqdm.tqdm], optional):
            The progress bar API, you can use put a :cls:`tqdm` object here.
            This function will update the progress bar description.
            Defaults to None.

    Returns:
        RandomizedEntangledEntropyComplex:
            A dictionary contains purity, entropy,
            a list of each overlap, puritySD, degree,
            actual measure range, bitstring range and more.
    """
```
This function returns a dictionary that contains the entropy, purity, and other relevant information. 
The returned `dict` contains the following information:

```python
class RandomizedEntangledEntropyComplex(TypedDict):
    """The result of the analysis."""

    purity: Union[np.float64, float]
    """The purity of the system."""
    entropy: Union[np.float64, float]
    """The entropy of the system."""
    purityCells: Union[dict[int, np.float64], dict[int, float]]
    """The purity of each cell."""
    puritySD: Union[np.float64, float]
    """The standard deviation of the purity."""
    entropySD: Union[np.float64, float]
    """The standard deviation of the entropy."""
    degree: Optional[Union[tuple[int, int], int]]
    """The range of partition."""
    measureActually: Union[tuple[int, int], tuple[()]]
    """The range of partition refer to all qubits."""
    bitStringRange: Union[tuple[int, int], tuple[()]]
    """The range of partition on the bitstring."""
    countsNum: int
    """The number of counts."""
```




---

#### Dummy Data


In [1]:
from qurry.capsule import quickRead

easy_dummy: dict[str, dict[str, int]] = quickRead("./easy-dummy.json")
large_dummy_list = [easy_dummy["0"] for _ in range(100)]

/Users/harui/opt/anaconda3/envs/quantumsphere11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

#### Simple Example


In [2]:
from qurry.process.randomized_measure import (
    randomized_entangled_entropy_v1,
    RandomizedEntangledEntropyComplex,
)

test_result_1_1_1 = randomized_entangled_entropy_v1(4096, large_dummy_list, 6)

In [3]:
from pprint import pprint

print("| result of randomized_entangled_entropy_v1 except for purityCells")
pprint({k: v for k, v in test_result_1_1_1.items() if k != "purityCells"})
# "purityCells" is too long we skip it here

print("| result of randomized_entangled_entropy_v1[purityCells]")
print(test_result_1_1_1["purityCells"][0])
print(test_result_1_1_1["purityCells"][1])

| result of randomized_entangled_entropy_v1 except for purityCells
{'bitStringRange': (2, 8),
 'countsNum': 100,
 'degree': 6,
 'entropy': -0.08786065308638322,
 'entropySD': 0.0,
 'measureActually': (0, 8),
 'purity': 1.0627930164337158,
 'puritySD': 0.0,
 'takingTime': 18.007}
| result of randomized_entangled_entropy_v1[purityCells]
1.0627930164337158
1.0627930164337158


---

#### Integration with your own progress bar


In [4]:
from tqdm import tqdm
import time

all_counts_progress_01 = tqdm(
    [
        (4096, large_dummy_list, 6),
        (4096, large_dummy_list, (2, 8)),
        (4096, large_dummy_list, 7),
        (4096, large_dummy_list, (0, 7)),
        (4096, large_dummy_list, (-2, 5)),
        (4096, large_dummy_list, (-5, -1)),
        (4096, large_dummy_list, (3, -2)),
    ],
    bar_format="| {desc} - {elapsed} < {remaining}",
)

test_result_1_1_2 = []
for tmp_shot, tmp_counts, tmp_partition in all_counts_progress_01:
    test_result_1_1_2.append(
        randomized_entangled_entropy_v1(
            tmp_shot, tmp_counts, tmp_partition, pbar=all_counts_progress_01
        )
    )
    time.sleep(0.5)

|  - 00:00 < ?

| Calculate specific degree 6. - 00:00 < ?

| Calculate specific degree 6. - 00:15 < 01:30

| Calculate specific degree (2, 8). - 00:15 < 01:30

| Calculate specific degree (2, 8). - 00:24 < 00:57

| Calculate specific degree 7. - 00:24 < 00:57     

Process SpawnPoolWorker-26:
Process SpawnPoolWorker-25:
Process SpawnPoolWorker-29:
| Calculate specific degree 7. - 00:30 < 01:15

Process SpawnPoolWorker-31:
Process SpawnPoolWorker-32:
Process SpawnPoolWorker-27:
Process SpawnPoolWorker-28:
Process SpawnPoolWorker-30:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/harui/opt/anaconda3/envs/quantumsphere11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/harui/opt/anaconda3/envs/quantumsphere11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/harui/opt/anaconda3/envs/quantumsphere11/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/Users/harui/opt/anaconda3/envs/quantumsphere11/lib/python3.11/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harui/opt/anaconda3/envs/quantumsphere11/lib/python3.11/site

rui/opt/anaconda3/envs/quantumsphere11/lib/python3.11/email/feedparser.py", line 175, in feed
    self._input.push(data)
  File "/Users/harui/opt/anaconda3/envs/quantumsphere11/lib/python3.11/email/feedparser.py", line 110, in push
    parts = self._partial.readlines()
            ^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
  File "/Users/harui/opt/anaconda3/envs/quantumsphere11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/harui/opt/anaconda3/envs/quantumsphere11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/harui/opt/anaconda3/envs/quantumsphere11/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/Users/harui/opt/anaconda3/envs/quantumsphere11/lib/python3.11/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harui/opt/anac

KeyboardInterrupt: 

---

#### Using Python backend

It will be slow. You'd better think twice before using it.


In [5]:
all_counts_progress_02 = tqdm(
    [
        (4096, large_dummy_list, 6),
        (4096, large_dummy_list, (2, 8)),
        (4096, large_dummy_list, 7),
        (4096, large_dummy_list, (0, 7)),
        (4096, large_dummy_list, (-2, 5)),
        (4096, large_dummy_list, (-5, -1)),
        (4096, large_dummy_list, (3, -2)),
    ],
    bar_format="| {desc} - {elapsed} < {remaining}",
)


test_result_1_1_2 = []
for tmp_shot, tmp_counts, tmp_partition in all_counts_progress_02:
    test_result_1_1_2.append(
        randomized_entangled_entropy_v1(
            tmp_shot,
            tmp_counts,
            tmp_partition,
            pbar=all_counts_progress_02,
            backend="Python",
        )
    )
    time.sleep(0.5)

| Calculate specific degree (3, -2). - 00:09 < 00:00 


---

### Post-Processing Speed-up Availablities and Version Info



In [6]:
from qurry.process.status import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurry version: 0.9.2.dev1
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust  
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes   
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes   
   - wavefunction_overlap ................... Yes    Depr.  Yes   
   - echo_cell .............................. Yes    Depr.  Error 
 - utils
   - randomized ............................. Yes    Depr.  Yes   
   - construct .............................. Yes    No     Yes   
   - dummy .................................. Yes    No     Yes   
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes   
 - magnet_square
   - magnsq_core ............................ Yes    No     No    
--------------------------------------------------------
   + Yes ...... Working normally.
   + Error .... Exception occurred.
   + No ....... Not suppo